In [1]:
from collections import defaultdict
import pandas as pd

In [8]:
import numpy as np

In [3]:
data = pd.read_csv('/Users/zhaojunran/Documents/Python_Urban_Perception_and_Visual_Character/MIT_Place_Pulse_archive/votes_clean.csv')

In [4]:
data.columns

Index(['Unnamed: 0', 'place_id_left', 'place_id_right', 'study_id', 'left',
       'right', 'voter_uniqueid', 'choice', 'study_question',
       'place_name_right', 'place_name_left', 'day', 'time', 'long_right',
       'lat_right', 'long_left', 'lat_left'],
      dtype='object')

In [5]:
data.head()

,Unnamed: 0,place_id_left,place_id_right,study_id,left,right,voter_uniqueid,choice,study_question,place_name_right,place_name_left,day,time,long_right,lat_right,long_left,lat_left
0,1,50e5e9c1d7c3df413b000000,50f38e44fdc9f065f0000855,50f62cb7a84ea7c5fdd2e458,513d573bfdc9f03587003120,50f5eb3ffdc9f065f00080f7,7681bc0ade864091a957524067c9254f,left,wealthier,Rio De Janeiro,Amsterdam,2014-04-20,21:17:31,-22.779996,-43.403705,52.382016,4.861142
1,2,50e5e9c1d7c3df413b000000,50f39092fdc9f065f000097d,5217c351ad93a7d3e7b07a64,513d57f9fdc9f0358700326b,50f5e700fdc9f065f0006ef0,e5939a746d544ce09c04d6453a5a1a3a,left,more beautiful,Sao Paulo,Amsterdam,2013-09-01,03:22:19,-23.638028,-46.601966,52.325979,4.880425
2,3,50e5e9c1d7c3df413b000000,50e734e9d7c3df413b000ebf,50f62c41a84ea7c5fdd2e454,513d57e5fdc9f03587003244,513d7b54fdc9f03587006b19,32c3fe6369814f398dee39f3a68daa4f,right,livelier,Atlanta,Amsterdam,2017-10-02,01:40:35,33.895489,-84.282779,52.358512,4.943212
3,4,50e5e9c1d7c3df413b000000,50f39436fdc9f065f0000acc,50f62ccfa84ea7c5fdd2e459,513d57bdfdc9f035870031fd,50f562e3fdc9f065f0005b45,90dc7520a28f4c58a090d14ec58cb181,equal,more depressing,Singapore,Amsterdam,2013-12-29,16:34:34,1.343338,103.961166,52.332149,4.861824
4,5,50e5e9c1d7c3df413b000000,50f38aaefdc9f065f0000562,50f62c41a84ea7c5fdd2e454,513d56fefdc9f035870030ad,50f61675beb2fed9100008bb,b06240448e62439194a30171f2d84091,right,livelier,Philadelphia,Amsterdam,2016-05-03,13:20:42,40.022366,-75.143560,52.308278,4.977296


place_id_left
左图的唯一ID（Google Street View 图像 ID）

place_id_right
右图的唯一ID

left, right
图像的原始编号（可能用于识别或定位）

place_name_left
左图对应的城市名称（如 Amsterdam）

place_name_right
右图对应的城市名称（如 Rio De Janeiro）

choice
参与者的选择：left 表示更“富裕”（在本例中）的是左图

study_question
感知比较的维度（如 wealthier、safer、livelier）

long_, lat_
每张图的经纬度坐标（用于地理定位）

如果是自己组数据的话,务必带上打分部分

In [11]:
# 一共有几个研究问题
data['study_question'].unique()

array(['wealthier', 'more beautiful', 'livelier', 'more depressing',
       'safer', 'more boring', nan], dtype=object)

'wealthier':城市的富裕程度

'more beautiful':城市的美丽程度

'livelier':城市的活力程度

'more depressing':城市的压抑程度

'safer':城市的安全程度

'more boring':城市的无聊程度

'nan':空项

### 评分生成:简单计数法

1. 按感知维度（study_question）筛选数据，比如选“safer”；
2. 统计每张图像“赢”的次数（choice 为left 的是左图赢，为 right 的是右图赢）；
3. 统计每张图像参与比较的总次数；
4. 评分 = 赢的次数 / 总次数，得分范围 0 到 1。

#### 逐步拆解(平行于函数)

平票不对评分有任何帮助,所以忽略

In [20]:
data_safer = data[data['study_question'] == 'safer']
data_safer.head()

,Unnamed: 0,place_id_left,place_id_right,study_id,left,right,voter_uniqueid,choice,study_question,place_name_right,place_name_left,day,time,long_right,lat_right,long_left,lat_left
6,7,50e5e9c1d7c3df413b000000,50e70a8cd7c3df413b000c72,50a68a51fdc9f05596000002,513d56befdc9f03587003035,513d7e3dfdc9f03587007377,eacb66f0a26f4cf1920def1b1d73ef9b,left,safer,New York,Amsterdam,2014-10-22,00:10:12,40.726136,-73.938586,52.309643,4.889399
9,10,50e5e9c1d7c3df413b000000,50f39092fdc9f065f000097d,50a68a51fdc9f05596000002,513d575bfdc9f03587003156,50f5e56bfdc9f065f00068bd,51c1b2044406462fae9c53afea1c6736,left,safer,Sao Paulo,Amsterdam,2018-02-02,13:55:10,-23.561633,-46.547778,52.327005,4.896014
17,18,50e5e9c1d7c3df413b000000,50e745a4d7c3df413b00127a,50a68a51fdc9f05596000002,513d5728fdc9f035870030fc,513cd42efdc9f03587001ebe,ed968919b3d3423780292205ae14ea60,left,safer,Cape Town,Amsterdam,2014-10-21,02:44:35,-33.967551,18.674882,52.330043,4.912867
22,23,50e5e9c1d7c3df413b000000,50f39725fdc9f065f0000c38,50a68a51fdc9f05596000002,513d5805fdc9f0358700327d,50f43b6efdc9f065f000309e,cf35a58cda82429d8aed6b42ccfdff63,right,safer,Taipei,Amsterdam,2015-07-23,07:56:42,25.083113,121.517018,52.345381,4.977363
26,27,50e5e9c1d7c3df413b000000,50e73766d7c3df413b00103f,50a68a51fdc9f05596000002,513d57fffdc9f03587003273,513f2e02fdc9f0358700db3f,aa276b61f51b4d8791089b081ec0569f,equal,safer,Berlin,Amsterdam,2015-02-12,13:24:48,52.539489,13.302840,52.299118,4.930368


In [26]:
left_wins = data_safer[data_safer['choice'] == 'left'].value_counts()
right_wins = data_safer[data_safer['choice'] == 'right'].value_counts()
total_wins = left_wins.add(right_wins,fill_value = 0)
total_counts = pd.concat([data_safer['left'],data_safer['right']]).value_counts()
total_counts

50f448abfdc9f065f0004161    87
50f42c43fdc9f065f000198c    81
513d699ffdc9f035870047c4    75
5185d4edfdc9f03fd5001448    71
50f5e56cfdc9f065f00068d7    58
                            ..
51413a00fdc9f049260055dc     1
513d6986fdc9f0358700476f     1
513f2ad0fdc9f0358700d5d2     1
513d3290fdc9f035870028cf     1
50f5ebccfdc9f065f000857a     1
Name: count, Length: 111389, dtype: int64

#### 定义成函数

In [16]:
def compute_simple_scores(df,study_question = 'safer'):
    df = df[df['study_question'] == study_question]

    # 赢的统计 | left和right标的都是唯一编号,用这个可以区分城市
    left_wins = df[df['choice'] == 'left']['left'].value_counts() # 左边赢的安全的城市个数,返回一个series
    right_wins = df[df['choice'] == 'right']['right'].value_counts() #  右边赢的安全的城市个数,返回一个series
    total_wins = left_wins.add(right_wins,fill_value = 0) # 两边的series加在一起
    # 这行代码将 left_win 和 right_win 两个 Series 对象相加。add 方法用于对两个 Series 对象进行逐元素相加操作。
    # fill_value = 0 参数表示当两个 Series 中某个索引只在其中一个 Series 中存在时，在另一个 Series 中对应位置的值用 0 填充，
    # 这样可以确保所有索引都能参与相加运算，最终得到一个包含左右两边所有唯一值出现总次数的 Series 对象。

    # 总比较次数统计
    total_counts = pd.concat([df['left'],df['right']]).value_counts()
    # 将 'left' 列和 'right' 列的数据合并，统计合并后各个值出现的次数，即每个元素总的比较次数

    # 简单评分
    scores = (total_wins / total_counts).fillna(0) # 计算简单评分，用每个元素总的赢的次数除以总的比较次数，缺失值用 0 填充
    scores = scores.reset_index().rename(columns = {'index':'place_id',0:f'{study_question}_score'})
    return scores

In [17]:
scores = compute_simple_scores(data,study_question = 'safer')

In [19]:
scores = scores.rename(columns = {'count':'simple_score'})
scores.head()

,place_id,simple_score
0,50e5f7d4d7c3df413b00056a,0.411765
1,50e5f7d4d7c3df413b00056b,0.300000
2,50e5f7d4d7c3df413b00056c,0.375000
3,50e5f7d4d7c3df413b00056d,0.555556
4,50e5f7d4d7c3df413b00056e,0.100000


### **评分生成 – Elo Rating**

**R_new = R_old + K * (Actual - Expected)**
- Expected 是根据双方当前评分计算的胜率；
- Elo 评分系统的一个基本假设是：两个评分差为 400 的对象，强者赢的概率大约是 10 倍于弱者
- Actual 是实际结果（胜 = 1，负 = 0）；
- K 是评分变化系数,控制评分变化的幅度/灵敏度：（通常设为 20-40), PlacePulse 等图像感知打分 32-64 。 数字越大对于胜负越敏感。
- R_new 新的得分 
- R_old 旧的得分

**基本原理：每张图像有一个初始分数（通常设为 1000），当两张图像比较时：**
- 胜者得分提升，败者得分下降。
- 分数变化由对手分数差决定（越强的对手被你赢了，你涨分越多）。

**优势：**
- 考虑了“对手强度”；
- 动态更新评分，适合大规模比较数据；
- 简单实现，不依赖概率建模。

In [35]:
def elo_score_update(score_A,score_B,result_A,K = 32): #result_A = 1 表示 A 赢，0 表示 A 输
    expected_A = 1 / (1 + 10 ** ((score_B-score_A)/400)) # B比A大的越多,A获胜的概率越小
    expected_B = 1 - expected_A

    score_A_new = score_A + K * (result_A - expected_A)
    score_B_new = score_B + K * ((1-result_A) - expected_B)

    return score_A_new, score_B_new

def compute_elo_ratings(df_vote,K = 32,base_score = 1000):
    
    elo_scores = defaultdict(lambda:base_score) # 创建了一个 defaultdict 对象，用于存储 Elo 分数
    # defaultdict 是 Python 标准库 collections 模块中的一个特殊字典类型，它在访问不存在的键时，不会抛出 KeyError 异常，而是会使用一个默认值。
    # lambda:base_score 是一个匿名函数，作为 defaultdict 的默认工厂函数。
    # 当尝试访问 elo_scores 中不存在的键时，这个匿名函数会被调用，返回 base_score 作为默认值。

    for _,row in df_vote.iterrows(): 
    # 用于遍历 Pandas 数据框 df_votes 每一行的循环结构,忽略了每一行的索引
    # df_votes.iterrows() 是 Pandas 数据框对象的一个方法，它会返回一个迭代器，该迭代器每次迭代都会产生一个包含两个元素的元组。
    # 元组的第一个元素是行的索引（通常是整数或字符串），第二个元素是一个 Pandas 的 Series 对象，表示当前行的数据。
    # 在这个循环中，使用了 _ 作为占位符来忽略行的索引。_ 是 Python 中常用的一种约定，表示这个变量不会在后续的代码中使用。
        A = row['left']
        B = row['right']
        # 从遍历的每一行中获得参与比较的左右两边城市id,目前称为A和B两个对象
        choice = row['choice']

        # 判断谁赢了
        result_A = 1 if choice == 'left' else 0

        elo_A = elo_scores[A] 
        elo_B = elo_scores[B]
        # elo_scores = {A:elo_A,B:elo_B}

        # 更新分数
        new_elo_A,new_elo_B = elo_score_update(elo_A,elo_B,result_A,K = K) 
        # 用函数elo_score_update计算并更新城市的分数:输入这一轮的胜负和两个城市的成绩,更新两个城市的新成绩
        elo_scores[A] = new_elo_A # 更新后的成绩替换字典中原有的成绩
        elo_scores[B] = new_elo_B # 更新后的成绩替换字典中原有的成绩

    return dict(elo_scores)

In [36]:
data_safe = data[data['study_question'] == 'safer']
elo_dict = compute_elo_ratings(data_safe,K = 32,base_score = 1000)

In [37]:
elo_dict

{'513d56befdc9f03587003035': 1047.1086477037838,
 '513d7e3dfdc9f03587007377': 987.5652778110197,
 '513d575bfdc9f03587003156': 1043.3553346288327,
 '50f5e56bfdc9f065f00068bd': 945.7171597731143,
 '513d5728fdc9f035870030fc': 932.1562060952099,
 '513cd42efdc9f03587001ebe': 929.3999762749273,
 '513d5805fdc9f0358700327d': 865.308557695137,
 '50f43b6efdc9f065f000309e': 939.7770576510435,
 '513d57fffdc9f03587003273': 1019.1581259741406,
 '513f2e02fdc9f0358700db3f': 1030.4998438288221,
 '513d57bcfdc9f035870031f9': 1069.5448674089353,
 '514138dbfdc9f04926005307': 1034.6939043248917,
 '513d579cfdc9f035870031bd': 1000.736306793522,
 '50f5e6c5fdc9f065f0006dde': 1026.840087169672,
 '513d567bfdc9f03587002fc8': 973.7000210539821,
 '514150d7fdc9f04926007e4a': 1021.9324947511051,
 '513d56b1fdc9f03587003020': 1107.9404884898574,
 '50f5e51afdc9f065f0006628': 925.8001633600537,
 '513e1eaffdc9f03587009d1c': 1048.2384951260783,
 '513d57b6fdc9f035870031ec': 1040.2781863683472,
 '513d9c6cfdc9f03587007d2d': 10

In [39]:
elo_scores = pd.DataFrame(list(elo_dict.items()),columns = ['place_id','elo_score'])
# list(elo_dict.items())：将 elo_dict.items() 返回的视图对象转换为列表。列表中的每个元素都是一个包含键值对的元组

In [41]:
elo_scores.head()

,place_id,elo_score
0,513d56befdc9f03587003035,1047.108648
1,513d7e3dfdc9f03587007377,987.565278
2,513d575bfdc9f03587003156,1043.355335
3,50f5e56bfdc9f065f00068bd,945.717160
4,513d5728fdc9f035870030fc,932.156206


#### 数据合并

根据相同的'place_id'把image_data和elo_scores连接在一起

In [62]:
image_data = pd.read_csv('20251027_imgs_data.csv')
image_data.head()

,Unnamed: 0,cluster_centers_rgb,color_ratios_raw,cluster_centers_hsv,matched_colors_raw,matched_colors,color_ratios_matched,filename,h_mean,h_std,s_mean,v_mean,v_max,place_id,simple_score,elo_score_x,elo_score_y
0,0,[[153 160 165]\n [ 22 80 129]\n [127 123 111]...,[0.34035833 0.03794167 0.32735833 0.1571 0...,"[array([103, 19, 165], dtype=uint8), array([1...","['gray', 'cyan', 'orange', 'gray', 'yellow']","['gray', 'cyan', 'orange', 'yellow']","[np.float64(0.49745833333333334), np.float64(0...",51415142fdc9f04926007f43.jpg,59.777342,42.281507,43.972375,146.078017,255,51415142fdc9f04926007f43,0.000000,968.956512,968.956512
1,1,[[ 36 35 26]\n [171 174 177]\n [ 93 96 66]...,[0.27780833 0.1731 0.21171667 0.1403 0...,"[array([27, 71, 36], dtype=uint8), array([105,...","['yellow', 'gray', 'yellow', 'orange', 'yellow']","['yellow', 'gray', 'orange']","[np.float64(0.6866), np.float64(0.1731), np.fl...",50f5ed9afdc9f065f0008e32.jpg,44.728267,37.648842,62.034000,94.703683,255,50f5ed9afdc9f065f0008e32,0.500000,996.751165,996.751165
2,2,[[153 148 121]\n [ 53 54 32]\n [214 215 209]...,[0.202775 0.20121667 0.1415 0.24424167 0...,"[array([ 25, 53, 153], dtype=uint8), array([ ...","['yellow', 'yellow', 'white', 'cyan', 'yellow']","['yellow', 'white', 'cyan']","[np.float64(0.6142583333333334), np.float64(0....",513e6050fdc9f0358700b395.jpg,52.645892,38.345215,70.778133,143.088600,255,513e6050fdc9f0358700b395,0.692308,1081.661041,1081.661041
3,3,[[ 68 72 69]\n [204 205 204]\n [157 159 158]...,[0.24099167 0.27510833 0.28238333 0.09298333 0...,"[array([68, 14, 72], dtype=uint8), array([ 60,...","['gray', 'white', 'gray', 'gray', 'yellow']","['gray', 'white', 'yellow']","[np.float64(0.6163583333333332), np.float64(0....",50f5eb28fdc9f065f0008096.jpg,59.708800,44.573643,29.268692,136.006450,255,50f5eb28fdc9f065f0008096,0.000000,984.000000,984.000000
4,4,[[ 52 54 33]\n [143 140 133]\n [203 205 204]...,[0.23179167 0.2211 0.0928 0.12643333 0...,"[array([33, 99, 54], dtype=uint8), array([ 21,...","['yellow', 'gray', 'white', 'gray', 'gray']","['yellow', 'gray', 'white']","[np.float64(0.23179166666666667), np.float64(0...",50f5ec32fdc9f065f00087a7.jpg,46.192183,39.487293,49.982667,126.712392,255,50f5ec32fdc9f065f00087a7,0.250000,925.638091,925.638091


In [63]:
image_data = image_data.drop(columns = ['Unnamed: 0'])
image_data.head()

,cluster_centers_rgb,color_ratios_raw,cluster_centers_hsv,matched_colors_raw,matched_colors,color_ratios_matched,filename,h_mean,h_std,s_mean,v_mean,v_max,place_id,simple_score,elo_score_x,elo_score_y
0,[[153 160 165]\n [ 22 80 129]\n [127 123 111]...,[0.34035833 0.03794167 0.32735833 0.1571 0...,"[array([103, 19, 165], dtype=uint8), array([1...","['gray', 'cyan', 'orange', 'gray', 'yellow']","['gray', 'cyan', 'orange', 'yellow']","[np.float64(0.49745833333333334), np.float64(0...",51415142fdc9f04926007f43.jpg,59.777342,42.281507,43.972375,146.078017,255,51415142fdc9f04926007f43,0.000000,968.956512,968.956512
1,[[ 36 35 26]\n [171 174 177]\n [ 93 96 66]...,[0.27780833 0.1731 0.21171667 0.1403 0...,"[array([27, 71, 36], dtype=uint8), array([105,...","['yellow', 'gray', 'yellow', 'orange', 'yellow']","['yellow', 'gray', 'orange']","[np.float64(0.6866), np.float64(0.1731), np.fl...",50f5ed9afdc9f065f0008e32.jpg,44.728267,37.648842,62.034000,94.703683,255,50f5ed9afdc9f065f0008e32,0.500000,996.751165,996.751165
2,[[153 148 121]\n [ 53 54 32]\n [214 215 209]...,[0.202775 0.20121667 0.1415 0.24424167 0...,"[array([ 25, 53, 153], dtype=uint8), array([ ...","['yellow', 'yellow', 'white', 'cyan', 'yellow']","['yellow', 'white', 'cyan']","[np.float64(0.6142583333333334), np.float64(0....",513e6050fdc9f0358700b395.jpg,52.645892,38.345215,70.778133,143.088600,255,513e6050fdc9f0358700b395,0.692308,1081.661041,1081.661041
3,[[ 68 72 69]\n [204 205 204]\n [157 159 158]...,[0.24099167 0.27510833 0.28238333 0.09298333 0...,"[array([68, 14, 72], dtype=uint8), array([ 60,...","['gray', 'white', 'gray', 'gray', 'yellow']","['gray', 'white', 'yellow']","[np.float64(0.6163583333333332), np.float64(0....",50f5eb28fdc9f065f0008096.jpg,59.708800,44.573643,29.268692,136.006450,255,50f5eb28fdc9f065f0008096,0.000000,984.000000,984.000000
4,[[ 52 54 33]\n [143 140 133]\n [203 205 204]...,[0.23179167 0.2211 0.0928 0.12643333 0...,"[array([33, 99, 54], dtype=uint8), array([ 21,...","['yellow', 'gray', 'white', 'gray', 'gray']","['yellow', 'gray', 'white']","[np.float64(0.23179166666666667), np.float64(0...",50f5ec32fdc9f065f00087a7.jpg,46.192183,39.487293,49.982667,126.712392,255,50f5ec32fdc9f065f00087a7,0.250000,925.638091,925.638091


In [64]:
image_data['place_id'] = image_data['filename'].astype(str).str[:-4]
image_data['place_id']

0         51415142fdc9f04926007f43
1         50f5ed9afdc9f065f0008e32
2         513e6050fdc9f0358700b395
3         50f5eb28fdc9f065f0008096
4         50f5ec32fdc9f065f00087a7
                    ...           
110682    5140db1bfdc9f04926003e88
110683    513d9d13fdc9f03587007f59
110684    513e6b92fdc9f0358700c04f
110685    513d586efdc9f035870032f5
110686    513d6e72fdc9f035870052ec
Name: place_id, Length: 110687, dtype: object

In [65]:
elo_id = elo_scores['place_id'].unique()
elo_id

array(['513d56befdc9f03587003035', '513d7e3dfdc9f03587007377',
       '513d575bfdc9f03587003156', ..., '50f42b80fdc9f065f0001448',
       '513cd826fdc9f03587001f9d', '50f42bb4fdc9f065f0001635'],
      dtype=object)

In [66]:
image_data = image_data.merge(elo_scores,on = 'place_id')
image_data.head()

,cluster_centers_rgb,color_ratios_raw,cluster_centers_hsv,matched_colors_raw,matched_colors,color_ratios_matched,filename,h_mean,h_std,s_mean,v_mean,v_max,place_id,simple_score,elo_score_x,elo_score_y,elo_score
0,[[153 160 165]\n [ 22 80 129]\n [127 123 111]...,[0.34035833 0.03794167 0.32735833 0.1571 0...,"[array([103, 19, 165], dtype=uint8), array([1...","['gray', 'cyan', 'orange', 'gray', 'yellow']","['gray', 'cyan', 'orange', 'yellow']","[np.float64(0.49745833333333334), np.float64(0...",51415142fdc9f04926007f43.jpg,59.777342,42.281507,43.972375,146.078017,255,51415142fdc9f04926007f43,0.000000,968.956512,968.956512,968.956512
1,[[ 36 35 26]\n [171 174 177]\n [ 93 96 66]...,[0.27780833 0.1731 0.21171667 0.1403 0...,"[array([27, 71, 36], dtype=uint8), array([105,...","['yellow', 'gray', 'yellow', 'orange', 'yellow']","['yellow', 'gray', 'orange']","[np.float64(0.6866), np.float64(0.1731), np.fl...",50f5ed9afdc9f065f0008e32.jpg,44.728267,37.648842,62.034000,94.703683,255,50f5ed9afdc9f065f0008e32,0.500000,996.751165,996.751165,996.751165
2,[[153 148 121]\n [ 53 54 32]\n [214 215 209]...,[0.202775 0.20121667 0.1415 0.24424167 0...,"[array([ 25, 53, 153], dtype=uint8), array([ ...","['yellow', 'yellow', 'white', 'cyan', 'yellow']","['yellow', 'white', 'cyan']","[np.float64(0.6142583333333334), np.float64(0....",513e6050fdc9f0358700b395.jpg,52.645892,38.345215,70.778133,143.088600,255,513e6050fdc9f0358700b395,0.692308,1081.661041,1081.661041,1081.661041
3,[[ 68 72 69]\n [204 205 204]\n [157 159 158]...,[0.24099167 0.27510833 0.28238333 0.09298333 0...,"[array([68, 14, 72], dtype=uint8), array([ 60,...","['gray', 'white', 'gray', 'gray', 'yellow']","['gray', 'white', 'yellow']","[np.float64(0.6163583333333332), np.float64(0....",50f5eb28fdc9f065f0008096.jpg,59.708800,44.573643,29.268692,136.006450,255,50f5eb28fdc9f065f0008096,0.000000,984.000000,984.000000,984.000000
4,[[ 52 54 33]\n [143 140 133]\n [203 205 204]...,[0.23179167 0.2211 0.0928 0.12643333 0...,"[array([33, 99, 54], dtype=uint8), array([ 21,...","['yellow', 'gray', 'white', 'gray', 'gray']","['yellow', 'gray', 'white']","[np.float64(0.23179166666666667), np.float64(0...",50f5ec32fdc9f065f00087a7.jpg,46.192183,39.487293,49.982667,126.712392,255,50f5ec32fdc9f065f00087a7,0.250000,925.638091,925.638091,925.638091


In [55]:
image_data.to_csv('20251027_rating_data.csv')